In [1]:
import warnings
warnings.filterwarnings( 'ignore' )
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
import sys

In [2]:
partition = 300

In [3]:
!{sys.executable} -m pip install pytorch-tabnet --upgrade


[notice] A new release of pip is available: 24.1.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [4]:
trainpath = f'../../../../data/top30groups/DNDF_noGeographic/train/train{partition}.csv'
testpath = f'../../../../data/top30groups/DNDF_noGeographic/test/test{partition}.csv'

traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

In [5]:
import torch
print(torch.cuda.is_available())

True


In [6]:
import torch
torch.cuda.empty_cache()


In [7]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

def split_data(dftrain, dftest):
    Xtrain = dftrain.drop(columns=['gname']).values.astype(float)
    Ytrain = dftrain['gname'].values
    Xtest = dftest.drop(columns=['gname']).values.astype(float)
    Ytest = dftest['gname'].values

    le = LabelEncoder()
    Ytrain = le.fit_transform(Ytrain)
    Ytest = le.transform(Ytest)

    #y_pred_decoded = model.label_encoder.inverse_transform(y_pred)
    y_true_decoded = le.inverse_transform(Ytest)

    return Xtrain, Ytrain, Xtest, Ytest, y_true_decoded, le



In [8]:
#from pytorch_tabnet.sklearn import TabNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor


def find_best_tabnet(Xtrain, Ytrain, n_iter=20):
    print("Starting TabNet grid search")
    print("CUDA available:", torch.cuda.is_available())
    model = TabNetClassifier(verbose=0, seed=42)

    param_dist = {
        'n_d': [8, 16, 24],
        'n_a': [8, 16, 24],
        'n_steps': [3, 4, 5],
        'gamma': [1.0, 1.3, 1.5],
        'lambda_sparse': [1e-4, 1e-3, 1e-2],
        'optimizer_params': [{'lr': 0.01}]
    }

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
        scoring='accuracy',
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    fit_params = {
    'max_epochs': 100,        # adjust as needed
    'batch_size': 64,        # lower if you get CUDA OOM
    'virtual_batch_size': 16, # smaller "sub-batches" to reduce memory usage
    'patience': 10,
    'drop_last': False
}

    random_search.fit(Xtrain, Ytrain, **fit_params)
    print("Best parameters:", random_search.best_params_)
    print("Best accuracy:", random_search.best_score_)

    return random_search.best_params_


In [9]:
torch.cuda.empty_cache()

In [10]:
Xtrain, Ytrain, Xtest, Ytest, Ytest_decoded, le = split_data(traindata, testdata)
best_tabnet_params = find_best_tabnet(Xtrain, Ytrain)


# Re-initialize TabNet with best params
final_model = TabNetClassifier(
    **{k: v for k, v in best_tabnet_params.items()},
    verbose=1,
    seed=42
)

fit_params = {
    'max_epochs': 100,        # adjust as needed
    'batch_size': 64,        # lower if you get CUDA OOM
    'virtual_batch_size': 16, # smaller "sub-batches" to reduce memory usage
    'patience': 10,
    'drop_last': False
}

# Retrain on full training data
final_model.fit(
    Xtrain, Ytrain, **fit_params
)



Starting TabNet grid search
CUDA available: True
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)
/opt/conda/lib/python3.11/site-packages/pytorch_tabnet/

Best parameters: {'optimizer_params': {'lr': 0.01}, 'n_steps': 5, 'n_d': 24, 'n_a': 16, 'lambda_sparse': 0.01, 'gamma': 1.3}
Best accuracy: 0.43952380952380954
epoch 0  | loss: 3.83523 |  0:00:02s
epoch 1  | loss: 3.31549 |  0:00:04s
epoch 2  | loss: 3.06448 |  0:00:06s
epoch 3  | loss: 2.82151 |  0:00:08s
epoch 4  | loss: 2.66618 |  0:00:10s
epoch 5  | loss: 2.61638 |  0:00:13s
epoch 6  | loss: 2.55451 |  0:00:15s
epoch 7  | loss: 2.51004 |  0:00:17s
epoch 8  | loss: 2.46983 |  0:00:19s
epoch 9  | loss: 2.41803 |  0:00:21s
epoch 10 | loss: 2.36097 |  0:00:23s
epoch 11 | loss: 2.27692 |  0:00:25s
epoch 12 | loss: 2.28455 |  0:00:27s
epoch 13 | loss: 2.23928 |  0:00:30s
epoch 14 | loss: 2.2749  |  0:00:32s
epoch 15 | loss: 2.21234 |  0:00:34s
epoch 16 | loss: 2.20604 |  0:00:36s
epoch 17 | loss: 2.15744 |  0:00:38s
epoch 18 | loss: 2.14389 |  0:00:40s
epoch 19 | loss: 2.14365 |  0:00:42s
epoch 20 | loss: 2.11713 |  0:00:44s
epoch 21 | loss: 2.07395 |  0:00:46s
epoch 22 | loss: 2.09055 |

In [11]:
from sklearn.metrics import accuracy_score
import os

# Predict class indices for test set
y_pred = final_model.predict(Xtest)
y_pred_decoded = le.inverse_transform(y_pred)

# Compute and print accuracy
acc = accuracy_score(Ytest, y_pred)
file_path = os.path.join("results", f"gtd{partition}.txt")

# Make sure the directory exists
os.makedirs("results", exist_ok=True)

# Write a string to the file
with open(file_path, "w") as file:
    file.write(str(acc))

In [12]:
import os

# Make sure the directory exists
os.makedirs("Results", exist_ok=True)

with open(f'Results/tabnet_{partition}', "w") as f:
    f.write(f"Accuracy: {acc*100:.2f}%\n")

In [13]:
print(classification_report(Ytest_decoded, y_pred_decoded))

                                                  precision    recall  f1-score   support

                          Abu Sayyaf Group (ASG)       0.25      0.63      0.36       210
        African National Congress (South Africa)       0.51      0.97      0.67       210
                                Al-Qaida in Iraq       0.45      0.50      0.47       210
        Al-Qaida in the Arabian Peninsula (AQAP)       0.43      0.31      0.36       210
                                      Al-Shabaab       0.31      0.55      0.40       210
             Basque Fatherland and Freedom (ETA)       0.63      0.77      0.69       210
                                      Boko Haram       0.43      0.40      0.42       210
  Communist Party of India - Maoist (CPI-Maoist)       0.68      0.81      0.74       210
       Corsican National Liberation Front (FLNC)       0.59      0.69      0.63       210
                       Donetsk People's Republic       0.44      0.60      0.51       210
Farabundo

In [14]:
def plot_confusion_matrix(y_true, y_pred, labels):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"Results/confusion_matrix_partition_{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [15]:
# Get all unique class labels from the truths
class_labels = np.unique(Ytest_decoded)

plot_confusion_matrix(Ytest_decoded, y_pred_decoded, labels=class_labels)



Saved confusion matrix for partition 300 to Results/confusion_matrix_partition_300.png
